In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as vdatasets
import torchvision.utils as vutils
import pickle
import os, shutil

torch.manual_seed(1)
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 텐서보드 포트 설정
port= '6006'

# 텐서보드 데이터 파일 초기화
try:
    shutil.rmtree('runs/')
except:
    pass

In [3]:
port

'6006'

In [4]:
from tensorboardX import SummaryWriter

writer= SummaryWriter(comment="-tensorboard-basic")

# Scalar

In [5]:
writer.add_scalar

<bound method SummaryWriter.add_scalar of <tensorboardX.writer.SummaryWriter object at 0x10316d3c8>>

In [6]:
for n_iter in range(100):
    s1= torch.rand(1)
    s2= torch.rand(1)
    writer.add_scalar('data/scalar1', s1[0], n_iter) # data grouping by 'slash'
    writer.add_scalars('data/scalar_group', {'xsinx':n_iter*np.sin(n_iter),
                                            'xcosx':n_iter*np.cos(n_iter),
                                            'arctan':np.arctan(n_iter)}, n_iter)
    

# Graph

In [7]:
model= nn.Sequential(nn.Linear(2,10), nn.Sigmoid(), nn.Linear(10,1))
test_inputs= torch.randn(10,2)
outputs= model(test_inputs)

writer.add_graph(model, outputs)

Error occurs, No graph saved


RuntimeError: size mismatch, m1: [10 x 1], m2: [2 x 10] at /Users/soumith/miniconda2/conda-bld/pytorch_1532623076075/work/aten/src/TH/generic/THTensorMath.cpp:2070

In [8]:
torch.onnx

<module 'torch.onnx' from '/Users/hyundai/anaconda/envs/pytorch/lib/python3.6/site-packages/torch/onnx/__init__.py'>

In [9]:
# demo.py

import torch
import torchvision.utils as vutils
import numpy as np
import torchvision.models as models
from torchvision import datasets
from tensorboardX import SummaryWriter

resnet18 = models.resnet18(False)
writer = SummaryWriter()
sample_rate = 44100
freqs = [262, 294, 330, 349, 392, 440, 440, 440, 440, 440, 440]

for n_iter in range(100):

    dummy_s1 = torch.rand(1)
    dummy_s2 = torch.rand(1)
    # data grouping by `slash`
    writer.add_scalar('data/scalar1', dummy_s1[0], n_iter)
    writer.add_scalar('data/scalar2', dummy_s2[0], n_iter)

    writer.add_scalars('data/scalar_group', {'xsinx': n_iter * np.sin(n_iter),
                                             'xcosx': n_iter * np.cos(n_iter),
                                             'arctanx': np.arctan(n_iter)}, n_iter)

    dummy_img = torch.rand(32, 3, 64, 64)  # output from network
    if n_iter % 10 == 0:
        x = vutils.make_grid(dummy_img, normalize=True, scale_each=True)
        writer.add_image('Image', x, n_iter)

        dummy_audio = torch.zeros(sample_rate * 2)
        for i in range(x.size(0)):
            # amplitude of sound should in [-1, 1]
            dummy_audio[i] = np.cos(freqs[n_iter // 10] * np.pi * float(i) / float(sample_rate))
        writer.add_audio('myAudio', dummy_audio, n_iter, sample_rate=sample_rate)

        writer.add_text('Text', 'text logged at step:' + str(n_iter), n_iter)

        for name, param in resnet18.named_parameters():
            writer.add_histogram(name, param.clone().cpu().data.numpy(), n_iter)

        # needs tensorboard 0.4RC or later
        writer.add_pr_curve('xoxo', np.random.randint(2, size=100), np.random.rand(100), n_iter)

dataset = datasets.MNIST('mnist', train=False, download=True)
images = dataset.test_data[:100].float()
label = dataset.test_labels[:100]

features = images.view(100, 784)
writer.add_embedding(features, metadata=label, label_img=images.unsqueeze(1))

# export scalar data to JSON for external processing
writer.export_scalars_to_json("./all_scalars.json")
writer.close()